In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sys

import pyspark

In [2]:
# read the new dev data
csv_folder_path = '/home/flame/Desktop/AML Project/amex-default-prediction/dev.csv/'

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(csv_folder_path) if file.endswith(".csv")]

# Read only the first 20% of data from each CSV file
dfs = []
for csv_file in csv_files:
    file_path = os.path.join(csv_folder_path, csv_file)
    total_rows = pd.read_csv(file_path, nrows=1).shape[0]
    skip_rows = int(0.8 * total_rows)  # Skip the first 80% of rows
    df = pd.read_csv(file_path, skiprows=range(1, skip_rows + 1))
    dfs.append(df)

# Concatenate the DataFrames
df = pd.concat(dfs, ignore_index=True)

In [3]:
# set options to see the dataframe in entirety
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
df.head(5)

,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,NaN,0.652984,0.008520,NaN,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,NaN,0.647093,0.002238,NaN,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975624,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.001034,0.00

In [5]:
df.shape

(1103628, 191)

In [6]:
# See if there any null values
v1 = df.isnull().sum()
v2 = ((df.isnull().sum()/len(df))*100).round(5)

[f'{percent:.2f}%' for percent in v2]

null_df = pd.DataFrame({'Null Value Count':v1,
                       'Null Value Percent':[f'{percent:.2f}%' for percent in v2]})
null_df

,Null Value Count,Null Value Percent
customer_ID,0,0.00%
target,0,0.00%
S_2,0,0.00%
P_2,9340,0.85%
D_39,0,0.00%
B_1,0,0.00%
B_2,435,0.04%
R_1,0,0.00%
S_3,203258,18.42%
D_41,435,0.04%


### so we can see that there are null values in the dataset. 
## Drop all the columns with more than 10% missing values

In [7]:
# calculate % if missing
v2 = df.isnull().mean()*100

# filter out columns with more than 10% nulls
cols_to_drop = v2[v2>10].index

# droppping these columns from the dataframe
df.drop(columns=cols_to_drop, inplace=True)

# recalculate nulls
v1 = df.isnull().sum()
v2 = ((df.isnull().sum()/len(df))*100).round(5)

[f'{percent:.2f}%' for percent in v2]

null_df2 = pd.DataFrame({'Null Value Count':v1,
                       'Null Value Percent':[f'{percent:.2f}%' for percent in v2]})
null_df2

,Null Value Count,Null Value Percent
customer_ID,0,0.00%
target,0,0.00%
S_2,0,0.00%
P_2,9340,0.85%
D_39,0,0.00%
B_1,0,0.00%
B_2,435,0.04%
R_1,0,0.00%
D_41,435,0.04%
B_3,435,0.04%


In [8]:
df.shape

(1103628, 152)

### We have already reduced the dataframe size greatly. We will perform imputation on a later stage after we are done with initial EDA

# Look the column types of the dataset

In [9]:
print(df.dtypes)

customer_ID     object
target           int64
S_2             object
P_2            float64
D_39           float64
B_1            float64
B_2            float64
R_1            float64
D_41           float64
B_3            float64
D_44           float64
B_4            float64
D_45           float64
B_5            float64
R_2            float64
D_47           float64
B_6            float64
B_7            float64
B_8            float64
D_51           float64
B_9            float64
R_3            float64
D_52           float64
P_3            float64
B_10           float64
S_5            float64
B_11           float64
S_6            float64
D_54           float64
R_4            float64
B_12           float64
S_8            float64
D_55           float64
B_13           float64
R_5            float64
D_58           float64
B_14           float64
D_59           float64
D_60           float64
B_15           float64
S_11           float64
D_63            object
D_64            object
D_65       

In [10]:
# there are some columns with object Dypes. Let us see which are these columns
num = []
cat = []

for col in df.columns:
    if df[col].dtype == 'object':
        cat.append(col)
    else:
        num.append(col)

print(num)
print('\n')
print(cat)

['target', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'D_41', 'B_3', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_47', 'B_6', 'B_7', 'B_8', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'B_12', 'S_8', 'D_55', 'B_13', 'R_5', 'D_58', 'B_14', 'D_59', 'D_60', 'B_15', 'S_11', 'D_65', 'B_16', 'B_18', 'B_19', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'P_4', 'D_74', 'D_75', 'B_24', 'R_7', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_30', 'S_18', 'D_86', 'R_17', 'R_18', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_103', 'D_104', 'D_107', 'B_36', 'B_37', 'R_27', 'B_38', 'D_109', 'D_112', 'B_40', 'D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 

In [11]:
# so, we have 3 columns of the categorical type. Let us look at these columns:
for col in cat:
    if col != 'customer_ID':
        print(col)
        print(df[col].nunique())
        print(df[col].value_counts())
        print('\n')

S_2
396
S_2
2018-02-17    5345
2018-01-20    4971
2018-02-24    4689
2018-02-03    4669
2018-03-17    4604
2017-11-18    4582
2018-03-03    4553
2018-01-27    4532
2018-02-10    4418
2018-03-24    4282
2018-03-10    4268
2018-01-13    4263
2017-08-19    4256
2017-09-16    4215
2017-12-16    4210
2017-10-21    4178
2017-11-11    4161
2017-12-09    4155
2017-11-25    4152
2017-08-12    4090
2017-04-15    4073
2017-09-23    4066
2017-07-15    4047
2017-05-20    4042
2017-10-14    4037
2017-06-17    3999
2017-06-03    3998
2017-06-24    3986
2017-07-22    3967
2017-07-29    3965
2017-12-23    3923
2017-04-08    3895
2017-08-26    3879
2017-10-28    3873
2017-09-09    3843
2017-04-22    3826
2017-05-27    3809
2017-11-27    3770
2017-03-25    3770
2017-05-13    3758
2017-07-08    3741
2017-03-18    3673
2017-06-10    3666
2018-02-27    3638
2018-03-20    3622
2018-02-22    3552
2017-12-21    3545
2017-12-20    3529
2018-02-16    3511
2017-10-20    3508
2017-11-24    3506
2018-02-15    3506


### okk, so columns D_63 and D_64 can be encoded using one-hot encoding. S_2 is the time component, so we don't need to encode it

In [12]:
customer_counts = df['customer_ID'].value_counts()

counts_df = pd.DataFrame(customer_counts).reset_index()
counts_df.columns = ['c_id', 'counts']

# grouping by counts to calculate number of customers
count_summary = counts_df.groupby('counts')['c_id'].nunique().reset_index()

# renaming the columns
count_summary.columns = ['count_bucket', 'num_customers']

# show dataframe
count_summary

,count_bucket,num_customers
0,1,1035
1,2,1208
2,3,1215
3,4,955
4,5,936
5,6,1075
6,7,1034
7,8,1247
8,9,1268
9,10,1342


### okk, so we have customers with data ranging from 13 months to 1 months

# Aggregation

In [13]:
# prefix_mapping = {'D_': 'Delinquency_',
#     'S_': 'Spend_',
#     'P_': 'Payment_',
#     'B_': 'Balance_',
#     'R_': 'Risk_'}


# for col in df.columns:
#     for prefix, full_name in prefix_mapping.items():
#         if col.startswith(prefix):
#             new_col_name = full_name + col[len(prefix):]
#             df.rename(columns={col: new_col_name}, inplace=True)

# print(df.columns)

In [14]:
# sorted_cols = sorted(df.columns)
# df = df[sorted_cols]

# print(df.columns)

# Feature Engineering

In [18]:
CAT_VARS = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_68"]

TARGET_COLUMN = 'target'

In [19]:
def feature_engineer(df, CAT_VARS=CAT_VARS):
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    
    cont_vars = [col for col in all_cols if col not in CAT_VARS]
    
    # Filter out non-numeric columns from continuous variables
    cont_vars_numeric = [col for col in cont_vars if df[col].dtype != 'object']
    
    # creating additional columns with aggregation of existing continuous var columns
    cont_vars_agg = df.groupby("customer_ID")[cont_vars_numeric].agg(['mean', 'std', 'min', 'max', 'last'])
    # renaming the new agg cols with _
    cont_vars_agg.columns = ['_'.join(x) for x in cont_vars_agg.columns]

    # Filter out non-numeric columns from categorical variables
    cat_vars_numeric = [col for col in CAT_VARS if df[col].dtype != 'object']
    cat_vars_agg = df.groupby("customer_ID")[cat_vars_numeric].agg(['count', 'last', 'nunique'])
    cat_vars_agg.columns = ['_'.join(x) for x in cat_vars_agg.columns]

    df = pd.concat([cont_vars_agg, cat_vars_agg], axis=1)
    del cont_vars_agg, cat_vars_agg
    print('shape after engineering', df.shape)
    
    return df

In [20]:
train_df = feature_engineer(df)

shape after engineering (91633, 724)


In [21]:
train_df.columns

Index(['target_mean', 'target_std', 'target_min', 'target_max', 'target_last',
       'P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', 'P_2_last',
       ...
       'D_117_nunique', 'D_120_count', 'D_120_last', 'D_120_nunique',
       'D_126_count', 'D_126_last', 'D_126_nunique', 'D_68_count', 'D_68_last',
       'D_68_nunique'],
      dtype='object', length=724)

In [ ]:
# X = df.drop('customer_ID', axis=1, inplace=True)

In [ ]:
X = df.drop('target', axis=1)
y = df.pop('target')